# Projeto: Identificador Unificado de Idiomas e Linguagens de Programação

Aluno: Pedro Henrique Madeira de Oliveira Pereira

Este projeto tem como objetivo desenvolver um modelo de NLP capaz de classificar textos em uma arquitetura unificada. O diferencial desta abordagem é a capacidade de distinguir não apenas idiomas humanos (como Português e Inglês), mas também linguagens de programação (como Python e Java) utilizando um único classificador.

A metodologia baseia-se na extração de n-grams de caracteres. Essa técnica captura a "textura" sintática e morfológica dos dados, permitindo diferenciar a estrutura de um código-fonte da gramática de uma língua natural sem a necessidade de modelos de Deep Learning pesados.

## Dependências

Nesta etapa, realizamos a importação das bibliotecas fundamentais que sustentam o projeto. Utilizamos o Pandas para a manipulação e análise estruturada dos dados e a biblioteca Datasets (da Hugging Face) para o acesso ágil aos repositórios de texto. O núcleo do processamento inteligente é provido pelo Scikit-learn, de onde extraímos as ferramentas essenciais para a construção do pipeline, treinamento do modelo e cálculo das métricas de avaliação.

In [ ]:
%pip install datasets pandas scikit-learn joblib

import pandas as pd
import joblib
from datasets import load_dataset

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

## Dataset de linguagens naturais

Aqui, é feito o carregamento do primeiro conjunto de dados, focado em identificação de idiomas naturais (papluca/language-identification). Após a conversão para um formato tabular (pandas), aplicamos um mapeamento para padronizar os rótulos das línguas (ex: de 'pt' para 'Portuguese'), garantindo clareza e consistência na etapa de classificação.

In [ ]:
print("Loading natural languages training dataset...")
dataset_natural = load_dataset("papluca/language-identification", split='train')

df_natural = dataset_natural.to_pandas()

df_natural = df_natural.rename(columns={'labels': 'language'})

lang_map = {
    'ar': 'Arabic',
    'bg': 'Bulgarian',
    'de': 'German',
    'el': 'Modern Greek',
    'en': 'English',
    'es': 'Spanish',
    'fr': 'French',
    'hi': 'Hindi',
    'it': 'Italian',
    'ja': 'Japanese',
    'nl': 'Dutch',
    'pl': 'Polish',
    'pt': 'Portuguese',
    'ru': 'Russian',
    'sw': 'Swahili',
    'th': 'Thai',
    'tr': 'Turkish',
    'ur': 'Urdu',
    'vi': 'Vietnamese',
    'zh': 'Chinese'
}

df_natural['language'] = df_natural['language'].map(lang_map).fillna(df_natural['language'])

print(f"Loaded {len(df_natural)} natural language examples.")
print(df_natural.head())

Loading natural languages training dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded 70000 natural language examples.
     language                                               text
0  Portuguese  os chefes de defesa da estónia, letónia, lituâ...
1   Bulgarian  размерът на хоризонталната мрежа може да бъде ...
2     Chinese  很好，以前从不去评价，不知道浪费了多少积分，现在知道积分可以换钱，就要好好评价了，后来我就把...
3        Thai  สำหรับ ของเก่า ที่ จริงจัง ลอง   honeychurch  ...
4     Russian                             Он увеличил давление .


## Dataset de linguagens de programação

Aqui é importado o conjunto de dados referente às linguagens de programação, proveniente do repositório Rosetta Code. Efetuamos a renomeação das colunas para que correspondam à estrutura do dataset anterior, permitindo a futura fusão dos dois universos de dados em um único fluxo de treinamento.

In [ ]:
print("Loading programming languages training dataset...")
dataset_prog = load_dataset("christopher/rosetta-code", split='train')

df_prog = dataset_prog.to_pandas()

df_prog = df_prog.rename(columns={'code': 'text', 'language_name': 'language'})

df_prog = df_prog[['text', 'language']]

print(f"Loaded {len(df_prog)} programming languages examples.")
print(df_prog.head())

Loading programming languages training dataset...
Loaded 79013 programming languages examples.
                                                text  language
0  BEGIN # find all primes with strictly increasi...  ALGOL 68
1  ascending?: function [x][\n    initial: digits...    Arturo
2   \n# syntax: GAWK -f ASCENDING_PRIMES.AWK\nBEG...       AWK
3   \n// Ascending primes. Nigel Galloway: April ...        F#
4  USING: grouping math math.combinatorics math.f...    Factor


## Combinação dos datasets

Combinamos os dados de linguagens naturais e de programação em um único DataFrame. Nesta fase, é feita uma limpeza crítica removendo classes que possuem apenas uma amostra (singletons), pois elas não fornecem dados suficientes para o aprendizado e poderiam introduzir ruído ou erros técnicos durante a divisão estratificada dos dados.

In [ ]:
print("Combining datasets...")
df_combined = pd.concat([df_natural, df_prog], ignore_index=True)

df_combined = df_combined.dropna()

class_counts = df_combined['language'].value_counts()

rare_classes = class_counts[class_counts == 1].index.tolist()

print(f"Dropping {len(rare_classes)} (of {len(class_counts)} total) classes with only one sample...")

df_combined = df_combined[~df_combined['language'].isin(rare_classes)]

df_combined = df_combined.reset_index(drop=True)

print(f"\nRemaining examples: {len(df_combined)}")
print("\nUpdated class distribution:")
print(df_combined['language'].value_counts())

print(f"Total of {len(df_combined)} combined examples.")

print("\nClasses distribution:")
print(df_combined['language'].value_counts())

Combining datasets...
Dropping 250 (of 1025 total) classes with only one sample...

Remaining examples: 148763

Updated class distribution:
language
Swahili               3503
French                3500
Vietnamese            3500
English               3500
Polish                3500
                      ... 
MINIL                    2
Extended BrainF***       2
Jack                     2
ferite                   2
Pict                     2
Name: count, Length: 775, dtype: int64
Total of 148763 combined examples.

Classes distribution:
language
Swahili               3503
French                3500
Vietnamese            3500
English               3500
Polish                3500
                      ... 
MINIL                    2
Extended BrainF***       2
Jack                     2
ferite                   2
Pict                     2
Name: count, Length: 775, dtype: int64


## Divisão de amostras de treino e teste

Aqui ocorre a divisão dos dados em subconjuntos de treinamento (80%) e teste (20%). Utilizamos a estratégia de estratificação para garantir que a proporção original de cada linguagem seja mantida em ambos os conjuntos, o que é crucial dado o desbalanceamento natural entre algumas classes.

In [ ]:
X = df_combined['text']
y = df_combined['language']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"\nTrain examples: {len(X_train)}")
print(f"Test examples: {len(X_test)}")


Train examples: 119010
Test examples: 29753


## Definição da pipeline

Aqui, definimos a arquitetura do modelo, que opera em dois passos. Primeiro, o TfidfVectorizer transforma o texto em representações numéricas analisando sequências de caracteres (n-grams), o que permite ao computador capturar a "textura" e a estrutura dos símbolos (como chaves em código ou acentos em texto) sem precisar entender o significado semântico. Em seguida, o classificador Naive Bayes (MultinomialNB) utiliza esses dados para calcular probabilisticamente a qual categoria aquele padrão textual pertence, uma abordagem reconhecida por ser leve e extremamente rápida.

In [ ]:
text_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        analyzer='char',
        ngram_range=(2, 5),
        min_df=5,
        max_features=200000
    )),
    ('clf', MultinomialNB()),
])

print("Model pipeline created.")
print(text_pipeline)

Model pipeline created.
Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer='char', max_features=200000, min_df=5,
                                 ngram_range=(2, 5))),
                ('clf', MultinomialNB())])


## Treinamento

Aqui ocorre o treinamento, onde o modelo aprende a associar os padrões de caracteres às respectivas linguagens. Graças à escolha de algoritmos eficientes, o modelo processa centenas de milhares de exemplos em poucos instantes, ajustando os pesos probabilísticos para as centenas de classes disponíveis.

In [ ]:
print("Starting model training...")
text_pipeline.fit(X_train, y_train)
print("Training finished.")

Starting model training...
Training finished.


## Avaliação

Avaliamos a performance do modelo utilizando o conjunto de dados de teste, que não foi visto durante o treinamento. A métrica de acurácia global e o relatório de classificação detalhado nos permitem entender não apenas o desempenho geral, mas também a precisão/precision e a revocação/recall específicas para cada linguagem.

In [ ]:
print("Evalutating model on the test set...")
y_pred = text_pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy * 100:.2f}%")

print("\nDetailed classification report:")
print(classification_report(y_test, y_pred))

Evalutating model on the test set...

Accuracy: 72.40%

Detailed classification report:


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                precision    recall  f1-score   support

                          0815       0.00      0.00      0.00         3
                           11l       1.00      0.80      0.89       157
                  360 Assembly       1.00      0.81      0.89        42
                            4D       0.00      0.00      0.00         1
                    4DOS Batch       0.00      0.00      0.00         1
                 6502 Assembly       0.00      0.00      0.00        21
                 6800 Assembly       0.00      0.00      0.00         1
                68000 Assembly       0.00      0.00      0.00        15
                 8051 Assembly       0.00      0.00      0.00         2
                 8080 Assembly       0.00      0.00      0.00        17
                 8086 Assembly       0.00      0.00      0.00        22
                           8th       0.00      0.00      0.00        18
              AArch64 Assembly       0.97      0.91      0.94  

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Serialização

Por fim, serializamos o modelo treinado em um arquivo binário (.joblib). Este passo é fundamental para a produtização do projeto, permitindo que o classificador seja carregado posteriormente por uma API sem a necessidade de realizar um novo treinamento a cada execução.

In [ ]:
model_filename = 'trained_model.joblib'
joblib.dump(text_pipeline, model_filename)

print(f"\nFinal model saved successfully at: {model_filename}")


Final model saved successfully at: trained_model.joblib
